In [1]:
from pynq import Overlay
from PIL import Image as PIL_Image
from PIL import ImageDraw as PIL_ImageDraw
from PIL import ImageFont
import matplotlib.pyplot as plt
import numpy as np
import math
import funct
import addresses
from pynq import MMIO
import cv2
from pynq import allocate
import time
import ctypes

In [2]:
ORIG_IMG_PATH   = 'dog.jpg'

In [3]:
overlay = Overlay("Yolo_test.bit")
overlay?

In [4]:
import gc
weight_base_buffer = allocate(shape=(25470896,), dtype=np.uint32, cacheable=False)
WEIGHT_BASE = weight_base_buffer.physical_address
print("100M",weight_base_buffer.physical_address)
params_wight = np.fromfile("weightsv2_comb_reorg_ap16.bin", dtype=np.uint32)
np.copyto(weight_base_buffer, params_wight)
print("yolov2_weight copy ok\n")
del weight_base_buffer
gc.collect()
del params_wight
gc.collect()


bate_base_buffer = allocate(shape=(5381,), dtype=np.uint32, cacheable=False)
print("32k",bate_base_buffer.physical_address)
BETA_BASE=bate_base_buffer.physical_address
params_bais = np.fromfile("biasv2_comb_ap16.bin", dtype=np.uint32)
np.copyto(bate_base_buffer, params_bais)
print("yolov2_bais copy ok\n")
del bate_base_buffer
gc.collect()
del params_bais
gc.collect()

img_base_buffer = allocate(shape=(4194304,), dtype=np.int32)
print("16M",img_base_buffer.physical_address)
IMG_MEM = img_base_buffer.physical_address



RuntimeError: Allocate failed: 4294967295

In [ ]:

import addresses

classes = 80
batch = 1
coords=4
background = 0
softmax =1

S=2
K=3 
Tn=4
Tm=32
Tr=26
Tc=26

OnChipIB_Width=((Tc-1)*S+K)
OnChipIB_Height=((Tr-1)*S+K)
ALPHA_BETA_MAX_NUM=1024

MEM_BASE = IMG_MEM

MEM_LEN = 416*416*32*2+208*208*32*2
Memory_top = MEM_BASE
Memory_bottom = MEM_BASE + MEM_LEN
    
ROUTE16_LEN =26*26*512*4//2
CONV27_LEN =13*13*256*4//2
CONV24_LEN =13*13*1024*4//2

LastLayerOutputPara = pow(2.0,-addresses.inputQ[23])

in_ptr  = np.zeros(32)
out_ptr = np.zeros(32)
for x in range(0,18):
        if x%2 == 0:
            in_ptr[x] = Memory_top
            out_ptr[x] =  Memory_bottom - addresses.net_layers_outputs[x]*4//2
        else:
            in_ptr[x] = out_ptr[x-1]
            out_ptr[x] = Memory_top
            

for x in range(18,25):
    if x%2 == 0:
            in_ptr[x] = Memory_top
            out_ptr[x] = Memory_bottom - ROUTE16_LEN - addresses.net_layers_outputs[x]*4//2
    
    else:
            in_ptr[x] = out_ptr[x-1];
            out_ptr[x] = Memory_top;
        
in_ptr[26] = Memory_bottom - ROUTE16_LEN
out_ptr[26] = Memory_top  

in_ptr[27] = Memory_top
out_ptr[27] = Memory_bottom - ROUTE16_LEN - CONV24_LEN - CONV27_LEN
    
in_ptr[29] = out_ptr[27]
out_ptr[29] = Memory_top

in_ptr[30] = Memory_top
out_ptr[30] = Memory_bottom - (addresses.net_layers_outputs[30] + 1024*7)*4//2

if(out_ptr[30]%(4*1024)!=0):
    out_ptr[30] = (out_ptr[30]//(4*1024)-1)*(4*1024)
in_ptr[31] = out_ptr[30]

del MEM_BASE
gc.collect()
del MEM_LEN 
gc.collect()
del CONV27_LEN 
gc.collect()
del ROUTE16_LEN 
gc.collect()
del CONV24_LEN
gc.collect()
del Memory_top
gc.collect()
del Memory_bottom
gc.collect()

In [ ]:

def load_image():
        orig_img_path = (ORIG_IMG_PATH)

        try:
            img = PIL_Image.open(orig_img_path)
            print("Open pictrue success!")
        except IOError:
            print("fail to load image!")
        print("pictrue size:",img.size)
        print("pictrue mode:",img.mode)  
        return img
    
def letterbox_image(img,w,h):
        if((float)(w/img_w))<((float)(h/img_h)):
            new_w = w
            new_h = (img_h*w)//img_w
        else:
            new_w = (img_w*h)//img_h
            new_h = h
        img_resized = img.resize((new_w,new_h),PIL_Image.BILINEAR)

        boxed = PIL_Image.new('RGB',(w,h),(127,127,127))
        box1 = (0,0,new_w,new_h)
        boxed.paste(img_resized.crop(box1),((w-new_w)//2,(h-new_h)//2))
        
        return boxed

   


In [ ]:


def yolo_fpga(img_base_buffer,region_buff):
         
    TR=0
    TC=0
    TM=0
    TN=0
    offset_index=0
    woffset = 0
    boffset = 0

    woffset = 0
    boffset = 0
    
    
    
    OUTPUT = np.zeros((36504,), dtype=np.int32)
    current_p = np.zeros((36504,), dtype=np.int16)
    next_p = np.zeros((36504,), dtype=np.int16)
    output_p = np.zeros((73008,), dtype=np.int16)
    region_buff_temp = np.zeros((73008,), dtype=np.float32)
    LastLayerOutputPara = math.pow(2.0,-addresses.inputQ[23])

    for i in range(0,32):
       
        if addresses.layertype[i] == 0:
            print(i,"conv")
            output_w = (addresses.layerw[i] - addresses.layersize[i] + 2*addresses.layerpad[i])//addresses.layerstride[i] + 1
            output_h = (addresses.layerh[i] - addresses.layersize[i] + 2*addresses.layerpad[i])//addresses.layerstride[i] + 1

            TR = min(((OnChipIB_Height-addresses.layersize[i])//addresses.layerstride[i]+1),Tr)                #keep Kernel_stride>=1
            TR = min(output_h,TR)
            TC = min(((OnChipIB_Width-addresses.layersize[i])//addresses.layerstride[i]+1),Tc)
            TC = min(output_w,TC)
            TM = min(addresses.layern[i],Tm)
            TN = min(addresses.layerc[i],Tn)

            rLoops =int(math.ceil(float(output_h)/TR))
            cLoops =int(math.ceil(float(output_w)/TC))
            mLoops =int(math.ceil(float(addresses.layern[i])/TM))
            nLoops =int(math.ceil(float(addresses.layerc[i])/TN))

            INPUTQ_t = addresses.inputQ[offset_index];
            if(i==26):
                INPUTQ_t = addresses.inputQ[12];
    
            funct.YOLO__Init_EX(int(in_ptr[i]),int(out_ptr[i]),woffset//2,boffset//2,
                addresses.layerc[i],addresses.layern[i],addresses.layersize[i],
                addresses.layerstride[i],addresses.layerw[i],addresses.layerh[i],output_w,output_h,
                addresses.layerpad[i],addresses.layeractivation[i],addresses.layerbatch_normalize[i],
                TM,TN,TR,TC,
                mLoops,nLoops,rLoops,cLoops,0,
                INPUTQ_t,addresses.inputQ[offset_index+1],addresses.weightQ[offset_index],addresses.betaQ[offset_index],
                WEIGHT_BASE,BETA_BASE)


    
            woffset += addresses.weight_offset[offset_index]
            boffset += addresses.beta_offset[offset_index]  
            offset_index = offset_index+1

        elif addresses.layertype[i] == 1:
            print(i,"maxpool")
            output_w = (addresses.layerw[i] - addresses.layersize[i])//addresses.layerstride[i] + 1 
            output_h = (addresses.layerh[i] - addresses.layersize[i])//addresses.layerstride[i] + 1  

            TR = min(((OnChipIB_Height-addresses.layersize[i])//addresses.layerstride[i]+1),Tr)  #//keep Kernel_stride>=1
            TR = min(output_h,TR)
            TC = min(((OnChipIB_Width-addresses.layersize[i])//addresses.layerstride[i]+1),Tc)
            TC = min(output_w,TC)
            TM = min(Tm,Tn)
            TM = min(addresses.layerc[i],TM)
            TN = TM

            rLoops = int(math.ceil(float(output_h)/TR))
            cLoops = int(math.ceil(float(output_w)/TC))
            mLoops = int(math.ceil(float(addresses.layerc[i])/TM))



            funct.YOLO__Init_EX(int(in_ptr[i]),int(out_ptr[i]),0,0,addresses.layerc[i],addresses.layerc[i],
                        addresses.layersize[i],addresses.layerstride[i],addresses.layerw[i],addresses.layerh[i],output_w,output_h,
                          0,0,0,TM,0,TR,TC,mLoops,1,rLoops,cLoops,1,
                        addresses.inputQ[offset_index],addresses.inputQ[offset_index],20,20,
                        WEIGHT_BASE,BETA_BASE)

        elif addresses.layertype[i] == 2:
            print(i,"reorg")
            TR = min(((OnChipIB_Height-addresses.layerstride[i])//addresses.layerstride[i]+1),Tr)      #//keep Kernel_stride>=1
            TC = min(((OnChipIB_Width-addresses.layerstride[i])//addresses.layerstride[i]+1),Tc)

            TM = 4
            TN = TM
            output_w = 26;
            output_h = 26*16;
            TR = min(output_h,TR)
            TC = min(output_w,TC)
            rLoops = int(math.ceil(float(output_h)/TR))
            cLoops = int(math.ceil(float(output_w)/TC))
            mLoops = 1


            funct.YOLO__Init_EX(int(in_ptr[i]),int(out_ptr[i]),0,0,1,4,
            addresses.layerstride[i],addresses.layerstride[i],52,52*16,output_w,output_h,
                          0,0,0,TM,TN,TR,TC,mLoops,1,rLoops,cLoops,2,
            addresses.inputQ[offset_index],addresses.inputQ[offset_index],20,20,
            WEIGHT_BASE,BETA_BASE)
        elif addresses.layertype[i] == 3:
            print(i,"route")
        elif addresses.layertype[i] == 4:
            print(i,"region")
            address_start = int((in_ptr[i] - MEM_BASE)//4)
            address_end   = int(address_start +13*13*432//2) 
            OUTPUT = img_base_buffer[address_start:address_end].copy()           
            current_p = OUTPUT%65536
            next_p    = OUTPUT//65536
            output_p[::2]  = current_p
            output_p[1::2] =  next_p 
            region_buff_temp = output_p*LastLayerOutputPara 
            np.copyto(region_buff,region_buff_temp)
    print("FPGA_Accelerate_Completed!!")

In [ ]:
class box:
    def __init__(self):
            self.x = float()
            self.y = float()
            self.w = float()
            self.h = float()

class detection:
    def __init__(self):
            self.bbox = box() 
            self.classes = int()
            self.prob = []
            self.mask = []
            self.objectness = float()
            self.sort_class = int()

def entry_index(outputs,w,h,batch,location,entry):
            n  =   location //(w*h)    
            loc = location % (w*h)
            return batch*outputs + n*w*h*(4+classes+1) + entry*w*h + loc


def get_region_box(x,biases,n,index,i,j,w,h,stride):

            b=box()
            b.x = (i + x[index + 0*stride]) / w
            b.y = (j + x[index + 1*stride]) / h
            b.w = math.exp(x[index + 2*stride]) * biases[2*n]   / w
            b.h = math.exp(x[index + 3*stride]) * biases[2*n+1] / h
            return b


def correct_region_boxes(n,w,h,netw,neth,dets):

            new_w=0
            new_h=0
    
            if ((float(netw)/w) < (float(neth)/h)):
                new_w = netw;
                new_h = (h * netw)//w
            else:
                new_h = neth
                new_w = (w * neth)//h

            for i in range(0,n):
                b = dets[i].bbox
                b.x =  (b.x - (netw - new_w)/2/netw) /  (float(new_w)/netw)
                b.y =  (b.y - (neth - new_h)/2/neth) /  (float(new_h)/neth)

                b.w = b.w *float(netw)/new_w
                b.h =  b.h *float(neth)/new_h
                
                relative  = 1 
                nrelative = 0
                if(nrelative):
                    b.x *= w
                    b.w *= w
                    b.y *= h
                    b.h *= h

                dets[i].bbox = b

In [ ]:
 
def activate_array(x,index,n): 
#ACTIVATION  LOGISTIC     
        for i in range(0,n):
            x[index+i] = 1/(1+ math.exp(-x[index+i])) 
            

def forward_region_layer(net_input,layer_output,layerw,layerh,layern,outputs):
       
        for b in range(0,batch):          
            for n in range(0,addresses.layern):
              
                index = entry_index(outputs, addresses.layerw,addresses.layerh,b, n*addresses.layerw*addresses.layerh,0)
                activate_array(layer_output,index, 2*addresses.layerw*addresses.layerh)
                index = entry_index(outputs, addresses.layerw,addresses.layerh,b, n*addresses.layerw*addresses.layerh, coords)
                
                if True: 
                    activate_array(layer_output,index,addresses.layerw*addresses.layerh)
                    index = entry_index(outputs, addresses.layerw,addresses.layerh,b, n*addresses.layerw*addresses.layerh, coords + 1)
        lib = ctypes.cdll.LoadLibrary("./softmax.so") 
        if (softmax):
            
            index = entry_index(outputs,addresses.layerw,addresses.layerh,0,0,coords+1)          
            net_input_temp = (ctypes.c_float*len(net_input[index:]))(*net_input[index:])
            layer_output_temp =(ctypes.c_float*len(layer_output[index:]))(*layer_output[index:])
            
            
            lib.softmax_cpu(net_input_temp,classes + background,  batch*addresses.layern, outputs//addresses.layern, addresses.layerw*addresses.layerh,  1, addresses.layerw*addresses.layerh, ctypes.c_float(1), layer_output_temp)
            layer_output[index:] = np.array(layer_output_temp)

In [ ]:

nms=0.45
thresh = 0.5
hier_thresh = .5

#====================
#make_network_boxes   dets
nboxes =  845 
def get_region_detections(net_input,w,h,layerw,layerh,layern,netw,neth,outputs,thresh,dets):
            # get_region_detections
            predictions = net_input
     
            for i in range(0,addresses.layerw*addresses.layerh):
                row = i//addresses.layerw
                col = i % addresses.layerw
                for n in range(0,addresses.layern): 
                    index = n*addresses.layerw*addresses.layerh+i 
                   

                    obj_index  = entry_index(outputs, addresses.layerw,addresses.layerh,0,n*addresses.layerw*addresses.layerh + i, coords)
                    box_index  = entry_index(outputs,addresses.layerw,addresses.layerh,0, n*addresses.layerw*addresses.layerh + i, 0)
                    mask_index = entry_index(outputs, addresses.layerw,addresses.layerh,0, n*addresses.layerw*addresses.layerh + i, 4)
                    scale = float(predictions[obj_index])
                    dets[index].bbox = get_region_box(predictions,addresses.layerbias, n, box_index, col, row, addresses.layerw, addresses.layerh, addresses.layerw*addresses.layerh)
                    if(scale > thresh):dets[index].objectness=scale 
                    else:dets[index].objectness=0
                
                    #===================   
                    if(dets[index].mask):
                        for j in range(0,coords-4):
                            dets[index].mask.append(net_output[mask_index + j*addresses.layerw*addresses.layerh])
                           
                    class_index = entry_index(outputs, addresses.layerw,addresses.layerh,0, n*addresses.layerw*addresses.layerh + i, coords+1)
                    #=================== 
                    if(dets[index].objectness):
                        for j in range(0,classes):
                            class_index = entry_index(outputs,addresses.layerw,addresses.layerh, 0, n*addresses.layerw*addresses.layerh + i,coords + 1 + j)
                            prob_num = scale*predictions[class_index]   
                            if(prob_num > thresh):dets[index].prob[j]=prob_num
                       
            correct_region_boxes(addresses.layerw*addresses.layerh*addresses.layern, w, h,netw,neth,dets)
           
            
            

def fill_network_boxes(net_input,w,h,layerw,layerh,layern,netw,neth,outputs,thresh,dets):   
        get_region_detections(net_input,w,h,addresses.layerw,addresses.layerh,addresses.layern,netw,neth,outputs,thresh,dets)


In [ ]:
from operator import itemgetter, attrgetter

def box_iou(a, b):
    return box_intersection(a, b)/box_union(a, b)
def overlap(x1, w1, x2, w2):

    l1 = x1 - w1/2
    l2 = x2 - w2/2
    if(l1 > l2):left = l2 
    else:left = l1
    r1 = x1 + w1/2
    r2 = x2 + w2/2
    if(r1 < r2):right = r1 
    else:right = r2
    return right - left

def box_intersection(a, b):

    w = overlap(a.x, a.w, b.x, b.w)
    h = overlap(a.y, a.h, b.y, b.h)
    if(w < 0 or h < 0): return 0
    area = w*h
    return area

def box_union(a, b):

    i = box_intersection(a, b);
    u = a.w*a.h + b.w*b.h - i;
    return u


def do_nms_sort(total,classes,thresh,dets):
    
    k = total-1
    swap = detection()
    temp = 0
    while(temp<k+1):

        if(dets[temp].objectness == 0):
                swap = dets[temp]
                dets[temp] = dets[k]
                dets[k] = swap
                k =k-1 
                temp = temp-1
        temp= temp+1
            
    total = k+1


    for c in range(0,classes):
        
        for i in range(0,total):
            dets[i].sort_class = c
        dets[0:5:1] = sorted(dets[0:5:1],key =lambda x:(x.prob[x.sort_class]), reverse=True) 

        for i in range(0,total): 
            if(dets[i].prob[c] == 0): 
                continue
          
            a = dets[i].bbox
            for j in range (i+1,total):
                
                b = dets[j].bbox
                if (box_iou(a, b) > thresh):
                    dets[j].prob[c] = 0


In [ ]:
orig_name_path = ("coco.names")
label_name = []   
label_read = open(orig_name_path) 
all_lines = label_read.readlines()	
for line in all_lines:
	label_name.append(line.strip())	
del orig_name_path
gc.collect()
del label_read
gc.collect()
del all_lines
gc.collect()

In [ ]:
def draw_detections(img_out,img_w,img_h,num,thresh, names,classes,dets):
    rgb_cnt = 0
    for i in range(0,num):
        labelstr = []
        class_t = -1
        for j in range(0,classes):
                if (dets[i].prob[j] > thresh):
                    
                    if (class_t < 0):
                        labelstr.append(names[j])
                        class_t = j;
                    else: 
                        labelstr.append(names[j])
                    print("name:", names[j], dets[i].prob[j]*100)             
          
        if(class_t >= 0):

            width = int(img_h * .006)
            offset = class_t*123457 % classes
            b = dets[i].bbox
 

            left  = int((b.x-b.w/2.)*img_w)
            right = int((b.x+b.w/2.)*img_w)
            top   = int((b.y-b.h/2.)*img_h)
            bot   = int((b.y+b.h/2.)*img_h)

            if(left < 0): left = 0
            if(right > img_w-1): right = img_w-1
            if(top < 0): top = 0
            if(bot > img_h-1): bot = img_h-1
                
            if(rgb_cnt ==0):
                RGB =  (255,0,0)    #rgb
            if(rgb_cnt ==1):
                RGB = (0,255,0)    #yellow
            if(rgb_cnt ==2):
                RGB = (255,255,0)   #green
            rgb_cnt = rgb_cnt+1     
            if(rgb_cnt ==3):
                RGB = (153,248,88)   
            if(rgb_cnt ==4):
                RGB = (255,83,26)
            if(rgb_cnt ==5):
                RGB = (255,71,209)
            if(rgb_cnt == 5):
                rgb_cnt =0
            else:
                rgb_cnt = rgb_cnt+1 
            
            if(len(labelstr[0]) >5):
                label_len = len(labelstr[0])*10
            else:
                label_len = len(labelstr[0])*13
            
            
            draw = PIL_ImageDraw.Draw(img_out)
            draw.line([(left,bot),(left,top),(right,top),(right,bot),(left,bot)], fill = RGB, width = width)
            draw.rectangle([(left-width/2,top-24),(left+label_len,top-width/2)], fill = RGB)
            
            ttfont = ImageFont.truetype('Arial.ttf',20)
            draw.text((left+2,top-24),labelstr[0], fill = (0,0,0),font=ttfont)
           


In [ ]:

def yolo_meminout(frame_in,img_w,img_h,frame_out):
    
    start_time = time.time()
    print("p1")

    img_boxed = letterbox_image(frame_in,208,208)
    print("p2")
    img_boxed.save("pictrue_boxed.jpg")

    print("p3")
    img_array_3_416_416 = funct.image_to_array_1dim(img_boxed,208,208)
    del img_boxed
    gc.collect()
    print("p4")
    
    input_tmp_img = funct.float32_int(img_array_3_416_416)
    del img_array_3_416_416
    gc.collect()
    print("p5")
    end_time = time.time()
    image_preprocess = end_time - start_time 
    
    
    
    #load image to memory start
    start_time = time.time()
    np.copyto(img_base_buffer[0:194688],input_tmp_img)
    print("yolov2_image copy ok\n")
    end_time = time.time()
    load_image_to_memory = end_time - start_time 
    #load image to memory end
    
    #start yolo_fpga
    #fpga_process_time start
    start_time = time.time()
    region_buff = np.zeros((73008,), dtype=np.float32)
  
    yolo_fpga(img_base_buffer,region_buff)
    end_time = time.time()
    fpga_process_time = end_time - start_time 
    #fpga_process_time end
    
    
    #region_layer_process_time start
    start_time = time.time()
    layer_output =np.zeros(71825)
    net_input=np.zeros(71825)
    np.copyto(net_input,region_buff[0:71825])
    np.copyto(layer_output,net_input)
    forward_region_layer(net_input,layer_output,addresses.layerw[31],addresses.layerh[31],addresses.layern[31],addresses.net_layers_outputs[31])
    np.copyto(net_input,layer_output)
    end_time = time.time()
    region_layer_process_time = end_time - start_time
    #region_layer_process_time end
    
    
    #post_process_time start 
    dets = []
    start_time = time.time()
    for i in range(0,845):
        dets.append(detection())
    for i in range(0,845):
            dets[i].prob = np.zeros(classes,dtype = "float")
    fill_network_boxes(net_input,img_w,img_h,addresses.layerw[31],addresses.layerh[31],addresses.layern[31],addresses.layerw[0],addresses.layerh[0],addresses.net_layers_outputs[31],thresh,dets)
    
    do_nms_sort(nboxes,classes,nms,dets)
    
    draw_detections(frame_out,img_w,img_h,nboxes,thresh,label_name,classes,dets)
    end_time = time.time()
    
    
    post_process_time  = end_time - start_time
    
    print("image_preprocess         :",image_preprocess)
    print("load image to memory time:",load_image_to_memory)
    print("fpga_process_time        :",fpga_process_time)
    print("region_layer_process_time:",region_layer_process_time)
    print("post_process_time        :",post_process_time)
    



In [ ]:

frame_in=load_image()
img_w =frame_in.size[0]
img_h =frame_in.size[1]

    #frame_in_img = PIL_Image.fromarray(frame_in) 

img_out = frame_in
yolo_meminout(frame_in,img_w,img_h,img_out)
print("done-5")
img_out 

In [ ]:
frame_in=load_image()
img_w =frame_in.size[0]
img_h =frame_in.size[1]

#frame_in_img = PIL_Image.fromarray(frame_in) 

img_out = frame_in
yolo_meminout(frame_in,img_w,img_h,img_out)
img_out 